In [7]:
# Step 1: Install necessary libraries
!pip install -q transformers datasets accelerate bitsandbytes torch evaluate rouge_score sentencepiece bert_score

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
import pandas as pd
import random
import evaluate # Hugging Face's library for NLP evaluation
import warnings
import time

# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore")

In [9]:
# Step 2: Authenticate with Hugging Face
# This is required to download gated models like Llama 3
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGING_FACE_TOKEN")
except Exception as e:
    print("Could not retrieve Hugging Face token. Please ensure it is stored as a Kaggle secret named 'HUGGING_FACE_TOKEN'.")
    # You can manually paste your token here for local testing if needed:
    # hf_token = "YOUR_HF_TOKEN"
    hf_token = None

In [10]:
# Step 3: Define Model and Dataset Identifiers
dataset_id = "tmnam20/ViMedAQA"
NUM_SAMPLES_TO_EVALUATE = 1
# Prefer chat variants that ship with proper chat templates
model_ids = [
    "sail/Sailor-4B-Chat",         
    "vilm/vinallama-2.7b-chat"     
]

# In case upstream code still passes the older/base IDs, map them here:
MODEL_ID_ALIAS = {
    "sail/Sailor-4B": "sail/Sailor-4B-Chat",
    "vilm/vinallama-2.7b": "vilm/vinallama-2.7b-chat",
}

# Step 4: Load and Prepare the Dataset
try:
    dataset = load_dataset(dataset_id, split="train")
    print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

    # Create a small, random, representative sample for evaluation
    random.seed(42) # for reproducibility
    random_indices = random.sample(range(len(dataset)), NUM_SAMPLES_TO_EVALUATE)
    eval_dataset = dataset.select(random_indices)

    print(f"Created a random evaluation set with {len(eval_dataset)} samples.")
except Exception as e:
    print(f"Failed to load the dataset. Error: {e}")
    eval_dataset = None

Dataset loaded successfully! Total samples: 39881
Created a random evaluation set with 1 samples.


In [11]:
# --- UPDATED: Step 1 - Define Bilingual Prompt Engineering Strategies ---
# Mode 1: Run all strategies defined in PROMPT_STRATEGIES (False)
# Mode 2: Run only the single, specified strategy for a targeted comparison (True)
USE_BEST_PROMPT_ONLY = True
BEST_STRATEGY_NAME = "Extract_VI" # Specify the prompt to use in Mode 2
generation_times = {}

PROMPT_STRATEGIES = {
    # Các chiến lược ban đầu của bạn
    "Direct_VI": "Sử dụng Ngữ cảnh sau để trả lời Câu hỏi.",
    "RolePlay_VI": "Bạn là một trợ lý y tế hữu ích. Hãy trả lời Câu hỏi CHỈ dựa vào Ngữ cảnh được cung cấp.",
    "Extract_VI": "Dựa vào Ngữ cảnh sau, trích xuất câu trả lời trực tiếp từ văn bản, không giải thích gì thêm.",
    "Current_Best_VI": (
        "Bạn là một chuyên gia y tế AI với nhiệm vụ trích xuất thông tin chính xác. "
        "Dựa CHỈ vào văn bản trong phần Ngữ cảnh dưới đây, hãy trả lời cho Câu hỏi. "
        "Câu trả lời của bạn phải ngắn gọn, đi thẳng vào vấn đề và không chứa bất kỳ thông tin nào không có trong văn bản. "
        "Không giải thích thêm."
    ),

    # Các chiến lược được thêm vào
    "Chain_of_Thought_VI": (
        "Dựa vào Ngữ cảnh sau, hãy suy nghĩ từng bước một để đưa ra câu trả lời cho Câu hỏi. "
        "Hãy trình bày rõ ràng các bước suy luận của bạn."
    ),
    "Few_Shot_VI_ViMedAQA": (
        "Dựa vào các Ví dụ sau đây, hãy trả lời Câu hỏi cuối cùng bằng cách trích xuất thông tin từ Ngữ cảnh được cung cấp.\n\n"
        "--- Ví dụ 1 ---\n"
        "Ngữ cảnh: Thuốc Biviantac được chỉ định để điều trị các trường hợp do tăng tiết acid quá mức như: - Khó tiêu, nóng rát hay đau vùng thượng vị. - Trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua. - Tăng độ acid, đau rát dạ dày. - Các rối loạn thường gặp trong những bệnh lý loét dạ dày tá tràng, thực quản.\n"
        "Câu hỏi: Biviantac có thể điều trị trướng bụng, đầy hơi không?\n"
        "Câu trả lời: Có, Biviantac có thể điều trị các tình trạng như trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua.\n\n"
        "--- Ví dụ 2 ---\n"
        "Ngữ cảnh: Thuốc Atorvastatin T.V Pharm được dùng đường uống.\n"
        "Câu hỏi: Tổng hợp các cách dùng hiệu quả để quản lý Atorvastatin T.V Pharm?\n"
        "Câu trả lời: Các cách thức dùng thuốc Atorvastatin T.V Pharm hiệu quả là sử dụng đường uống.\n\n"
        "--- Ví dụ 3 ---\n"
        "Ngữ cảnh: - Buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột. - Mệt mỏi. - Ban, mày đay. - Thiếu máu tan huyết. - Yếu cơ. - Khó thở, sốc phản vệ.\n"
        "Câu hỏi: Các tác dụng phụ thường gặp của thuốc Aspirin 81 là gì?\n"
        "Câu trả lời: Các tác dụng phụ thường gặp của thuốc Aspirin 81 bao gồm buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột.\n\n"
        "--- Bây giờ, hãy trả lời Câu hỏi sau dựa trên Ngữ cảnh của nó---\n"
    ),
    "Expert_Persona_VI": (
        "Bạn là một chuyên gia trong lĩnh vực y tế."
        "Dựa trên kiến thức chuyên môn của mình, hãy trả lời Câu hỏi sau CHỈ dựa vào Ngữ cảnh được cung cấp."
    ),
}

def _resolve_model_id(model_id: str) -> str:
    return MODEL_ID_ALIAS.get(model_id, model_id)
    
def create_prompt(sample, model_id, tokenizer, strategy_name):
    """
    Create a fairly-compared prompt with model-specific chat formatting.
    - Sailor (Qwen1.5-based): roles = ["system", "question"]
    - VinaLLaMA Chat (ChatML): roles = ["system", "user"]
    """
    context = sample["context"]
    question = sample["question"]

    # 1) Instruction text
    base_instruction = PROMPT_STRATEGIES.get(strategy_name)
    if not base_instruction:
        raise ValueError(f"Strategy '{strategy_name}' not found in PROMPT_STRATEGIES.")
    full_instruction_text = f"{base_instruction}\n\nNgữ cảnh: {context}\n\nCâu hỏi: {question}"

    # 2) Model-specific formatting
    mid = _resolve_model_id(model_id)

    # --- Sailor-*-Chat (Qwen1.5 family): expects 'system' + 'question' ---
    if mid.startswith("sail/Sailor") and mid.endswith("-Chat"):
        system_prompt = "Bạn là một trợ lý hữu ích, trả lời CHỈ dựa vào Ngữ cảnh."
        messages = [
            {"role": "system", "content": system_prompt},
            # IMPORTANT: Sailor examples use role 'question' instead of 'user'
            {"role": "question", "content": full_instruction_text},
        ]
        return tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

    # --- VinaLLaMA 2.7B-Chat (ChatML): 'system' + 'user' ---
    if mid.startswith("vilm/vinallama-2.7b-chat"):
        system_prompt = "Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác."
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_instruction_text},
        ]
        return tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

    # Fallback: try generic LLaMA/Qwen-style roles (system+user)
    messages = [
        {"role": "system", "content": "Bạn là một trợ lý hữu ích."},
        {"role": "user", "content": full_instruction_text},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [12]:
# Step 5: Generate Answers from Each Model
all_generated_answers = {}

if eval_dataset and hf_token:
    # Wrap each answer in a list to create the required List[List[str]] structure
    ground_truth_answers = [[sample['answer']] for sample in eval_dataset] 
    questions = [sample['question'] for sample in eval_dataset]

    wide_results = []
    for i, sample in enumerate(eval_dataset):
        wide_results.append({
            "Sample_ID": i,
            "Question": sample['question'],
            "Context": sample['context'],
            "Ground_Truth_Answer": ground_truth_answers[i][0]
        })
    
    # Loop through each model to generate answers
    for raw_model_id in model_ids:
        model_id = _resolve_model_id(raw_model_id)  # dùng alias sang biến thể -Chat nếu cần
        print("\n" + "="*50)
        print(f"Loading model: {model_id} (requested: {raw_model_id})")
        print("="*50)

        model, tokenizer, text_generator = None, None, None

        try:
            # Load the tokenizer and model with 4-bit quantization to save memory
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=False,
            )
            tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token, trust_remote_code=True)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                token=hf_token,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True
            )

            # Set up the text generation pipeline
            text_generator = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )

            # Chọn danh sách chiến lược prompt
            if USE_BEST_PROMPT_ONLY:
                prompt_variations = [BEST_STRATEGY_NAME]
                print(f"Mode: Best Prompt Only. Running with the fair strategy: '{BEST_STRATEGY_NAME}'")
            else:
                prompt_variations = list(PROMPT_STRATEGIES.keys())
                print(f"Mode: Exploration. Running all {len(prompt_variations)} fair strategies: {prompt_variations}")

            # Thiết lập token kết thúc/đệm an toàn
            eos_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
            pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else eos_id

            for prompt_name in prompt_variations:
                print(f"\n--- Testing Prompt Strategy: {prompt_name} ---")
                result_key = f"{raw_model_id} ({prompt_name})"

                # Tạo toàn bộ prompts bằng hàm factory đã chuẩn hóa định dạng theo model
                prompts = [create_prompt(sample, model_id, tokenizer, prompt_name) for sample in eval_dataset]
                print(prompts)

                start_time = time.time()
                print(f"Generating answers for {len(prompts)} prompts using {model_id} with '{prompt_name}' strategy...")

                # Generate answers cho cả batch
                generated_outputs_batch = text_generator(
                    prompts,
                    max_new_tokens=256,
                    do_sample=False,             # greedy
                    eos_token_id=eos_id,
                    pad_token_id=pad_id,
                )
                end_time = time.time()
                generation_time = end_time - start_time
                generation_times[result_key] = generation_time
                print(f"Time for generating answer: {generation_time:.2f} seconds.")
    
                # Extract the clean answers bằng cách cắt phần completion sau prompt
                model_answers = []
                for i, output in enumerate(generated_outputs_batch):
                    generated_text = output[0]["generated_text"]
                    prompt_text = prompts[i]
                    if generated_text.startswith(prompt_text):
                        clean_answer = generated_text[len(prompt_text):].strip()
                    else:
                        # fallback rất hiếm khi cần (do pipeline đôi khi không echo full prompt)
                        clean_answer = generated_text.replace(prompt_text, "").strip()
                    model_answers.append(clean_answer)
    
                # Lưu kết quả
                all_generated_answers[result_key] = model_answers
                print(f"Successfully generated answers for {result_key}.")

                # Thêm cột vào wide_results
                answer_column_name = f"Answer_{result_key}"
                for i in range(len(model_answers)):
                    wide_results[i][answer_column_name] = model_answers[i]

        except Exception as e:
            print(f"An error occurred while processing {model_id}: {e}")
        finally:
            # Dọn tài nguyên
            if model is not None: del model
            if tokenizer is not None: del tokenizer
            if text_generator is not None: del text_generator
            torch.cuda.empty_cache()
            import gc; gc.collect()

else:
    print("Skipping generation due to issues with the dataset or Hugging Face token.")


Loading model: sail/Sailor-4B-Chat (requested: sail/Sailor-4B-Chat)


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Mode: Best Prompt Only. Running with the fair strategy: 'Extract_VI'

--- Testing Prompt Strategy: Extract_VI ---
['<|im_start|>system\nBạn là một trợ lý hữu ích, trả lời CHỈ dựa vào Ngữ cảnh.<|im_end|>\n<|im_start|>question\nDựa vào Ngữ cảnh sau, trích xuất câu trả lời trực tiếp từ văn bản, không giải thích gì thêm.\n\nNgữ cảnh: Thuốc D-Cure là sản phẩm có chứa hàm lượng vitamin D, giúp bổ sung vitamin D cho cơ thể. Sản phẩm này có tác dụng điều trị cũng như giúp dự phòng các bệnh cụ thể như: - Tình trạng thiếu hụt vitamin D.\n- Bị bệnh loãng xương. \n\nCâu hỏi: D-Cure có tác dụng dự phòng những bệnh gì?<|im_end|>\n<|im_start|>answer\n']
Generating answers for 1 prompts using sail/Sailor-4B-Chat with 'Extract_VI' strategy...
Time for generating answer: 25.76 seconds.
Successfully generated answers for sail/Sailor-4B-Chat (Extract_VI).

Loading model: vilm/vinallama-2.7b-chat (requested: vilm/vinallama-2.7b-chat)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


Mode: Best Prompt Only. Running with the fair strategy: 'Extract_VI'

--- Testing Prompt Strategy: Extract_VI ---
An error occurred while processing vilm/vinallama-2.7b-chat: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating


In [13]:
# --- Bước 5.5 - Lưu tất cả các câu trả lời đã tạo vào một tệp CSV ---
import pandas as pd
from IPython.display import display
import datetime

if wide_results:
    print("\n" + "="*50)
    print("Đang lưu kết quả định dạng rộng vào tệp CSV...")
    print("="*50)
    
    # Chuyển đổi danh sách kết quả thành một DataFrame
    results_df_wide = pd.DataFrame(wide_results)
    
    # Đặt tên file có timestamp để tránh bị ghi đè
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_path = f"/kaggle/working/generation_results_wide_{timestamp}.csv"
    
    # Lưu DataFrame vào tệp CSV
    results_df_wide.to_csv(output_file_path, index=False, encoding="utf-8-sig")
    
    print(f"Hoàn tất! Đã lưu {len(results_df_wide)} hàng (mẫu) vào tệp:")
    print(output_file_path)
    print("Các cột trong file CSV:")
    print(results_df_wide.columns.tolist())
    
    # Hiển thị 10 hàng đầu tiên để xem nhanh
    display(results_df_wide.head(10))
else:
    print("\nKhông có kết quả nào để lưu.")



Đang lưu kết quả định dạng rộng vào tệp CSV...
Hoàn tất! Đã lưu 1 hàng (mẫu) vào tệp:
/kaggle/working/generation_results_wide_20250913_164709.csv
Các cột trong file CSV:
['Sample_ID', 'Question', 'Context', 'Ground_Truth_Answer', 'Answer_sail/Sailor-4B-Chat (Extract_VI)']


,Sample_ID,Question,Context,Ground_Truth_Answer,Answer_sail/Sailor-4B-Chat (Extract_VI)
0,0,D-Cure có tác dụng dự phòng những bệnh gì?,Thuốc D-Cure là sản phẩm có chứa hàm lượng vit...,D-Cure có tác dụng dự phòng tình trạng thiếu h...,Sản phẩm D-Cure có tác dụng dự phòng tình trạn...


In [14]:
# Step 6: Evaluate the Generated Answers
if all_generated_answers:
    import evaluate
    import pandas as pd

    # Load metrics
    rouge_metric = evaluate.load("rouge")
    bleu_metric = evaluate.load("bleu")
    meteor_metric = evaluate.load("meteor")
    bertscore_metric = evaluate.load("bertscore")

    evaluation_results = []

    # --- Chuẩn hóa dữ liệu đầu vào cho các metric ---
    # predictions: List[str]
    # ground_truth_answers: List[List[str]] (đã có dạng này từ Step 5)
    # -> riêng cho BERTScore cần List[str], nên flatten tham chiếu
    references_for_bert = [refs[0] if isinstance(refs, list) and len(refs) > 0 else "" 
                           for refs in ground_truth_answers]

    print("\n" + "="*50)
    print("Calculating Evaluation Metrics")
    print("="*50)

    for result_key, predictions in all_generated_answers.items():
        print(f"\n--- Evaluating {result_key} ---")

        # Làm sạch outputs để tránh lỗi metric do None/ khoảng trắng thừa
        predictions = [("" if p is None else str(p)).strip() for p in predictions]

        # Check empty predictions
        if not any(predictions):
            print(f"  WARNING: Model & Prompt Strategy '{result_key}' produced empty answers for all samples. Assigning all metric scores to 0.")
            result_row = {
                "Model & Prompt Strategy": result_key,
                "ROUGE-L": 0.0,
                "BLEU": 0.0,
                "METEOR": 0.0,
                "BERTScore-F1": 0.0,
                "Generation Time (s)": round(generation_times.get(result_key, 0), 2),
            }
            evaluation_results.append(result_row)
            continue

        # Align lengths nếu số câu trả lời != số ground truths
        if len(predictions) != len(ground_truth_answers):
            min_len = min(len(predictions), len(ground_truth_answers))
            print(f"  NOTE: Mismatch sizes (pred={len(predictions)}, refs={len(ground_truth_answers)}). Truncating to {min_len}.")
            predictions = predictions[:min_len]
            refs_nested = ground_truth_answers[:min_len]
            refs_bert = references_for_bert[:min_len]
        else:
            refs_nested = ground_truth_answers
            refs_bert = references_for_bert

        # Tính các metric
        try:
            rouge_scores = rouge_metric.compute(predictions=predictions, references=refs_nested)
        except Exception as e:
            print(f"  ERROR computing ROUGE: {e}")
            rouge_scores = {"rougeL": 0.0}

        try:
            bleu_scores = bleu_metric.compute(predictions=predictions, references=refs_nested)
        except Exception as e:
            print(f"  ERROR computing BLEU: {e}")
            bleu_scores = {"bleu": 0.0}

        try:
            meteor_scores = meteor_metric.compute(predictions=predictions, references=refs_nested)
        except Exception as e:
            print(f"  ERROR computing METEOR: {e}")
            meteor_scores = {"meteor": 0.0}

        try:
            bertscore_scores = bertscore_metric.compute(
                predictions=predictions,
                references=refs_bert,     # <- BERTScore cần list[str]
                lang="vi"
            )
            bert_f1 = sum(bertscore_scores.get("f1", [])) / max(1, len(bertscore_scores.get("f1", [])))
        except Exception as e:
            print(f"  ERROR computing BERTScore: {e}")
            bert_f1 = 0.0

        # Tổng hợp kết quả
        result_row = {
            "Model & Prompt Strategy": result_key,
            "ROUGE-L": round(float(rouge_scores.get("rougeL", 0.0)), 4),
            "BLEU": round(float(bleu_scores.get("bleu", 0.0)), 4),
            "METEOR": round(float(meteor_scores.get("meteor", 0.0)), 4),
            "BERTScore-F1": round(float(bert_f1), 4),
            "Generation Time (s)": round(generation_times.get(result_key, 0), 2),
        }
        evaluation_results.append(result_row)

    # Step 7: Display Results
    results_df = pd.DataFrame(evaluation_results)
    results_df = results_df.sort_values(by="BERTScore-F1", ascending=False).reset_index(drop=True)

    print("\n--- Comparative Evaluation Results ---")
    try:
        from IPython.display import display
        display(results_df)
    except Exception:
        print(results_df.to_string(index=False))
else:
    print("\nNo answers were generated. Skipping evaluation.")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...



Calculating Evaluation Metrics

--- Evaluating sail/Sailor-4B-Chat (Extract_VI) ---


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


--- Comparative Evaluation Results ---


,Model & Prompt Strategy,ROUGE-L,BLEU,METEOR,BERTScore-F1,Generation Time (s)
0,sail/Sailor-4B-Chat (Extract_VI),0.1379,0.0617,0.4459,0.6724,25.76
